# Test 6

should be answered by building and calibrating a 10-period Black-Derman-Toy model for the short-rate.

Period 1 2 3 4 5 6 7 8 9 10

Spot Rate 3.0% 3.1% 3.2% 3.3% 3.4% 3.5% 3.55% 3.6% 3.65% 3.7%

Assume that $Z^6_0 = \frac{100}{(1+r_6)^6}$


In [1]:
SPOT_RATES = [.03, .031, .032, .033, .034, .035, .0355, .036, .0365, .037]
TEST_RATES = [.073, .0762, .081, .0845, .092, .0964, .1012, .1045, .1122, .1155, .1192, .122, .1232]

In [2]:
from utils import *

In [3]:
bdt = BlackDermanToy(SPOT_RATES, b=0.05)
a = bdt.calibrate()

In [4]:
rates, ep = bdt.calculate_spot_rates()
print(bdt.verify_correctness())

True


Once your model has been calibrated, compute the price of a payer swaption with notional $1M$ that expires at time $t=3$ with an option strike of $0$. You may assume the underlying swap has a fixed rate of $3.9\%$ and that if the option is exercised then cash-flows take place at times $t=4,…,10$. (The cash-flow at time $t=i$ is based on the short-rate that prevailed in the previous period, i.e. the payments of the underlying swap are made in arrears.)

In [5]:
""" Using short rates and knowledge that this is a payer swaption, 
    we can calculate the value of the swap until time t=4 
    Option owner doesn't receive/make coupon payments, from there we do regular option
    lattice reduction"""
fixed_rate = 0.039
short_rate_lattice = bdt.short_rates

In [6]:
swap_lattice = reduce_swap(fixed_rate, short_rate_lattice, start=0, periods=9, qu=.5, qd=.5, debug=False)
values = [[max(x, 0)for x in swap_lattice[3]]]

call = reduce_call(values, swap_lattice, short_rate_lattice, n=3, K=0, qu=.5, qd=.5, american=False, debug=False)

In [7]:
print(f"swaption value is ${round(call[0][0]*1_000_000,0)}")

swap value is $4085.0


# Q2

In [8]:
bdt = BlackDermanToy(SPOT_RATES, b=0.1)
bdt.calibrate()
rates, ep = bdt.calculate_spot_rates()
short_rate_lattice = bdt.short_rates

swap_lattice = reduce_swap(fixed_rate, short_rate_lattice, start=0, periods=9, qu=.5, qd=.5, debug=False)
values = [[max(x, 0)for x in swap_lattice[3]]]

call = reduce_call(values, swap_lattice, short_rate_lattice, n=3, K=0, qu=.5, qd=.5, american=False, debug=False)

print(f"swaption value is ${round(call[0][0]*1_000_000,0)}")

swap value is $8027.0


# Q3
assume 1-step hazard rate  is $h_{ij}=ab^{j-\frac{i}{2}}$
Compute the price of a zero-coupon bond with face value 
$F=100$ and recovery $R=20\%$.

In [9]:
n = 10
a = .01
b = 1.01
F = 100   # face value
R = .2    # recovery

srl = generate_short_rate_lattice(r00=.05 , n=n, u=1.1, d=0.9)

def h(i, j, a=a, b=b):
    """Get the hazard rate given some i and j"""
    return a*b**(j-i/2)

start out with the face value. then start reducing using pricing model for defaultable ZCB with recovery

get all the none default values ($\eta = 0$) since default values ($\eta = 1$)

In [10]:

def price_defaultable_zcb(face_value, periods, srl, R, q_u=0.5, q_d=0.5):
    zcb_matrix = [[100]*(periods + 1)]
    
    for i in range(n)[::-1]:
        last_prices = zcb_matrix[0]
        new_prices = []

        for j in range(i + 1):
            disc = 1/(1+srl[i][j])
            hij = h(i,j)   # the hazard rate at time i state j
            z_ij = disc*(q_u*(1-hij)*last_prices[j + 1] +
                         q_d*(1-hij)*last_prices[j] +
                         q_u*(hij)*R +
                         q_d*(hij)*R
                        )
            new_prices.append(z_ij)

        zcb_matrix.insert(0, new_prices)
    print(zcb_matrix)
    print("zcb length: ", len(zcb_matrix))
    return zcb_matrix[0][0]

print(round(price_defaultable_zcb(100, 10, srl, R*100), 2))

[[57.20919746903972], [57.9636042147442, 62.985198497360265], [59.06771119776974, 64.06267192376382, 68.50670364652537], [60.60805659931669, 65.52738500565765, 69.89438102260529, 73.71451915763441], [62.69657701429858, 67.47482393790962, 71.70273186510305, 75.38988702904797, 78.56967192002112], [65.48041904279022, 70.02904327818283, 74.0344198232338, 77.51338972523712, 80.50463532560194, 83.05083978746038], [69.15556189070782, 73.35143756298763, 77.02246380858972, 80.19363002134179, 82.90854219010775, 85.21202475886616, 87.15161329893668], [73.9889540899853, 77.65739752528661, 80.84012855664548, 83.56990533271255, 85.89398345512038, 87.8569233335553, 89.50371518605145, 90.87885419090993], [80.35186274056939, 83.23861665595004, 85.71730305547686, 87.82336349229206, 89.60415377473593, 91.09985140189374, 92.34830348244631, 93.38736850080653, 94.24664544215398], [88.76914845204672, 90.497123829278, 91.96286546490379, 93.19342062646933, 94.22511516263822, 95.08546528599668, 95.7986150493668

# Q4 

Solve for hazard rates given some markets bond prices. Use same hazard rate for every period to take advantage of risk neutral dynamics of interest rates. meaning discount for some period $k$ is $Z_0^{t_k}=d(0,t_n)$

In [11]:
F = 100
r = 0.05

In [12]:
# actual values
bond_prices = [100.92, 91.56, 105.6, 98.90, 137.48]
coupons = [.05, .02, .05, .05, .1]
recovery = [.1, .25, .5, .1, .2]

In [13]:
# example values
test_bond_prices = [101.20, 92.60, 107.44, 104.10, 145.84]
test_coupons = [.05, .02, .05, .05, .1]
test_recovery = [.1, .25, .5, .1, .2]

In [14]:
# months = 12*years
# time = [i for i in range(0, months+1, 6)]
survival_prob = [1.0]
discount_rates = [1.0]

In [15]:
from sympy.solvers import solve
from sympy import *
from scipy.optimize import minimize, broyden1
import numpy as np

def calibrate_hazard_rate(true_prices, coupon_rates, recovery_rates, interest_rate=0.05, F=100, debug=False):
    """ Returns hazard rates calibrated from true_prices of 
    Defaultable bonds with recovery. Assuming that the hazard rate is stable
    through out the year.
    """
    t = 0
    q = [1]
    default_prob = [None]
    years = len(true_prices)
    periods = years*2
    
    # discount rates for each semi annual period not including 1
    d = [round(1/(1+interest_rate/2)**i, 6) for i in range(1, periods + 1)]
    if debug: print("discount ", d)
    
    # fill out the list of hazard rates
    hazards = []        # reflects number of periods (6mth)
    hazard_unique = []  # reflects number of years
    for i in range(years):
        h_i = symbols(f'h_{i}')
        hazards.append(h_i)
        hazards.append(h_i)
        hazard_unique.append(h_i)

    if debug: print(hazards)

    # buld the survival probs
    for i in range(periods):
        q.append(q[i]*(1 - hazards[i]))
        
    
    def gen_bond_model(n, cr, rr, F=100, debug=debug):
        """ n is the period that the bond ends in
            cr is the coupon rate
            rr is the recovery rate
            F is the face value
        """
        eq = 0
        c = cr*F
        R = rr*F
        if debug: print(R)
        if debug: print(c)
        for i in range(n):
            if debug: print(f"i: {i}")
            # c*q_i*d(0,t_i)
            coup = c*q[i + 1]*d[i]
            if debug: print(f"used d: {d[i]} and q: {q[i+1]}")
            eq += coup
            if debug: print(f'coup: {c}*({q[i + 1]})*{d[i]}')
            
            # RF * (q_i-1 - q_i)*d(0, t_i)
            recov = R*(q[i] - q[i+1])*d[i]
            eq += recov
            if debug: print(f"recov: {R}*({q[i]} - {q[i+1]})*{d[i]}")
            
        # F*q_n*d(0,t_n)
        if debug: print(f"using n {n} to index")
        face = F*q[n]*d[n-1]
        if debug: print(f"Face = {face} = {F}*{q[n]}*{d[n-1]}")
        eq += face

        return eq

    err_eq = []
    n = 2
    bond_eq = []
    for tp, cr, rr in zip(true_prices, coupon_rates, recovery_rates):
        bond = gen_bond_model(n=n, cr=cr, rr=rr, debug=debug)
        bond_eq.append(bond)
        err_eq.append((bond - tp)**2)
        n += 2

    error = sum(err_eq)     
    def f(X):
        sub_d = {h_: x_ for h_, x_ in zip([h for h in hazard_unique], [x/100 for x in X])}
        res = error.subs(sub_d)

        return res

    l = []
    for i in range(4):
        l.append({'type':'ineq', 'fun': lambda x : x[i+1] - x[i]})
    cons = tuple(l)
    
    res = minimize(f, [2]*years, constraints=cons, method='BFGS')
    if debug: print(res.x)    
    
    i = 0
    sub_d = {h_: x_ for h_, x_ in zip([h for h in hazard_unique], [x/100 for x in res.x])}
    for be in bond_eq:
        if debug: print(be.subs(sub_d))

    return res.x
  

In [16]:
hazards = calibrate_hazard_rate(bond_prices, coupons, recovery)
test_hazards = calibrate_hazard_rate(test_bond_prices, test_coupons, test_recovery)

    

/Users/marcmatvienko/miniconda3/envs/phobos/lib/python3.8/site-packages/scipy/optimize/_minimize.py:521: RuntimeWarning: Method BFGS cannot handle constraints nor bounds.
  warn('Method %s cannot handle constraints nor bounds.' % method,


In [17]:
hazards

array([2.12378904, 2.62042742, 3.12157615, 3.62352235, 4.09913348])

# Q5

Compute par spread in basis points for a 5yr CDS with notional $N=\$10mil$ assuming recovery is $R=25\%$, the 3 month hazard rate is $1\%$ and interest is $5\%$ per annum.

All values are risk-neutral:
Premium payments value: $\Sigma_{k=1}^n\delta SNq(t_k)d(0,t_k)$

Val of accrued interest if default: $\frac{\delta SN}{2}\Sigma_{k=1}^n(q(t_{k-1}) - q(t_k))d(0,t_k)$

combined premium and interest: $\frac{\delta SN}{2}\Sigma_{k=1}^n(q(t_{k-1}) + q(t_k))d(0,t_k)$

val of contingent payment: $(1-R)N\Sigma_{k=1}^n(q(t_{k-1}) - q(t_k))d(0,t_k)$

par spread: val of contingent / combined premium and interest val

In [18]:
def get_cds_par_spread(N, R, q, h, periods=8):
    """ N : notional
        R : recovery rate
        q : interest rate (per annum)
        h : hazard rates (per quarter) 
        
        h is assumed to be fixed for simplicity sake
    """
    contingent = 0
    premium = 0   # includes accrued interest if default happens in middle
    
    survival = 1.0
    for i in range(periods + 1):
        discount = (1 + q/4)**-(i+1)
        old_surv = survival
        survival = survival*(1-h)
        
        # calc contingent
        contingent += (old_surv - survival)*discount
        # calc premium
        premium += (old_surv + survival)*discount
        
    contingent *= (1-R)
    premium *= 0.25/2
    
    return contingent / premium

In [19]:
# test get_cds_par_spread
get_cds_par_spread(10_000_000, .25, h=.01, q=.05)*100*100

301.5075376884429